In [4]:
import rioxarray
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [8]:
fn = '/Users/pibonacic/Documents/GitHub/poza-fria/data/ABI-L2-LST2KMF-2025-190-03-OR_ABI-L2-LST2KMF-M6_G19_s20251900300203_e20251900309511_c20251900314525.nc'

In [9]:
try:
    da = rioxarray.open_rasterio(fn, masked=True)
except rioxarray.errors.NotGeoreferencedWarning:
    # La advertencia se manejará aquí, pero el código continuará
    pass

/opt/miniconda3/envs/poza-fria/lib/python3.13/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/miniconda3/envs/poza-fria/lib/python3.13/site-packages/rioxarray/_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/opt/miniconda3/envs/poza-fria/lib/python3.13/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/miniconda3/envs/poza-fria/lib/python3.13/site-packages/rioxarray/_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
 

In [10]:
print("--- Información Completa del DataArray ---")
print(da)

--- Información Completa del DataArray ---
<xarray.Dataset> Size: 353MB
Dimensions:                 (band: 1, x: 5424, y: 5424)
Coordinates:
  * band                    (band) int64 8B 1
  * x                       (x) float64 43kB 0.5 1.5 2.5 ... 5.422e+03 5.424e+03
  * y                       (y) float64 43kB 0.5 1.5 2.5 ... 5.422e+03 5.424e+03
    goes_imager_projection  int64 8B 0
Data variables:
    DQF                     (band, y, x) float32 118MB ...
    LST                     (band, y, x) float32 118MB ...
    PQI                     (band, y, x) float32 118MB ...
Attributes: (12/253)
    algorithm_dynamic_input_data_container_input_ABI_L1b_FPT_from_focal_plane...
    algorithm_dynamic_input_data_container_input_ABI_L1b_FPT_from_focal_plane...
    algorithm_dynamic_input_data_container_input_ABI_L2_4_level_cloud_mask_da...
    algorithm_dynamic_input_data_container_input_ABI_L2_aerosol_optical_depth...
    algorithm_dynamic_input_data_container_input_ABI_L2_auxiliary_solar_ze

In [11]:
print("\n--- Nombres de las Coordenadas Disponibles ---")
print(list(da.coords.keys()))


--- Nombres de las Coordenadas Disponibles ---
['band', 'x', 'y', 'goes_imager_projection']


In [12]:
# MÉTODO 1: La forma ideal con rioxarray (Probablemente fallará aquí)
# Si el archivo estuviera georreferenciado correctamente, esto funcionaría.
# Dado el warning, es probable que devuelva (1.0, -1.0), que es un valor por defecto.
try:
    resolution = da.rio.resolution()
    print(f"\n--- Resolución según rioxarray (Método 1) ---")
    print(f"Resolución (x, y): {resolution}")
    print("(Nota: Si ves (1.0, -1.0), significa que rioxarray no encontró un geotransform)")
except Exception as e:
    print(f"\n--- rioxarray.rio.resolution() falló ---")
    print(f"Error: {e}")



--- Resolución según rioxarray (Método 1) ---
Resolución (x, y): (1.0, 1.0)
(Nota: Si ves (1.0, -1.0), significa que rioxarray no encontró un geotransform)


In [13]:
# MÉTODO 2: Calcular la resolución manualmente desde las coordenadas
# Esta es la forma más robusta cuando no hay geotransform.
print("\n--- Resolución calculada manualmente (Método 2) ---")

# Asumimos que las coordenadas espaciales se llaman 'x' e 'y'.
# ¡Debes verificar sus nombres reales imprimiendo 'da' arriba!
if 'x' in da.coords and 'y' in da.coords:
    # Extraemos los valores de las coordenadas
    x_coords = da['x'].values
    y_coords = da['y'].values

    # Calculamos la diferencia entre valores consecutivos
    # Usamos np.abs para obtener un valor positivo
    x_res = np.abs(np.diff(x_coords))
    y_res = np.abs(np.diff(y_coords))

    # Si la resolución es constante, todos los valores en x_res/y_res serán iguales.
    # Usamos la media para tener un solo número representativo.
    avg_x_res = np.mean(x_res)
    avg_y_res = np.mean(y_res)

    # Obtenemos las unidades de las coordenadas desde sus atributos
    x_units = da['x'].attrs.get('units', 'unidades desconocidas')
    y_units = da['y'].attrs.get('units', 'unidades desconocidas')

    print(f"Resolución en X: {avg_x_res:.6f} {x_units}")
    print(f"Resolución en Y: {avg_y_res:.6f} {y_units}")
else:
    print("No se encontraron las coordenadas 'x' e 'y' para calcular la resolución.")


--- Resolución calculada manualmente (Método 2) ---
Resolución en X: 1.000000 unidades desconocidas
Resolución en Y: 1.000000 unidades desconocidas


In [16]:
da.attrs['LST_resolution']

'y: 0.000056 rad x: 0.000056 rad'

In [17]:
da.attrs['spatial_resolution']

'2.0km at nadir'

In [18]:
da.attrs['goes_imager_projection_long_name']

'GOES-R ABI fixed grid projection'

Coordinates:
  * band                    (band) int64 8B 1
  * x                       (x) float64 43kB 0.5 1.5 2.5 ... 5.422e+03 5.424e+03
  * y                       (y) float64 43kB 0.5 1.5 2.5 ... 5.422e+03 5.424e+03
    goes_imager_projection  int64 8B 0
